## Encriptado Asimétrico

El cifrado asimétrico utiliza dos claves: una privada y otra pública. Las claves públicas se entregan para que cualquiera pueda utilizarlas, las conviertes en información pública. Cualquiera puede cifrar datos con tu clave pública y sólo quien tenga la clave privada puede descifrar el mensaje. 

También funciona a la inversa: puedes cifrar un mensaje con tu clave privada y cualquiera que tenga la pública lo podrá leer.

En cualquier caso, es importante mantener tu clave privada en secreto.

### Clave asimétrica desde programa

Existen dos librerías Python para trabajar con claves asimétricas: Pyca/cryptografy (https://cryptography.io/en/latest/) y PyCrypto (https://github.com/pycrypto/pycrypto)


### Ejemplo completo

Usaremos cryptography que hay que instalar 

`pip install cryptography`

#### Creación de claves

Puede hacerse con programas externos como OpenSSH u otros, pero la creamos desde programa

In [1]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives import serialization

privateKey = rsa.generate_private_key(public_exponent=65537, key_size=2048)
publicKey = privateKey.public_key()

ppk = privateKey.private_bytes(encoding=serialization.Encoding.PEM, format=serialization.PrivateFormat.PKCS8, encryption_algorithm=serialization.NoEncryption() )
f = open('claveCE.ppk', 'wb')
f.write(ppk)
f.close()

pem = publicKey.public_bytes(encoding=serialization.Encoding.PEM, format=serialization.PublicFormat.SubjectPublicKeyInfo)
f= open('claveCE.pem', 'wb')
f.write(pem)
f.close()

print("Par de claves generado con éxito")

Par de claves generado con éxito


#### Encriptado con clave pública

Si tenemos una clave pública disponible podemos usarla para encriptar nuestro contenido.





In [2]:
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import padding

pubKeyFile = open("claveCE.pem", "rb")
publicKey = serialization.load_pem_public_key(pubKeyFile.read(), backend=default_backend())

msjOriginal = "Saben que ya será inútil volver, que se les ha degradado la memoria y que de ahora en adelante vivirán como fantasmas parciales que no dejan huellas de sus pasos y carecen de sombra."
msjBytes = msjOriginal.encode()

pad = padding.OAEP(mgf=padding.MGF1(algorithm=hashes.SHA256()), algorithm=hashes.SHA256(), label=None)
msjEncriptado = publicKey.encrypt(msjBytes, pad)

print(msjEncriptado)

b'\x16\xaf\xbfG\x17\xc2\xe41\xfd\xa7\x7f3G\xa4\xcc\x1b\n\xb3\xeaj\xdf\x12\x84\xd6s\xbc\x1c\xd40\x98=\x1fu\xef\xc2M\xaf\x9c\x91\xf1\x96\xae\xb2\xf54\x9c8V\x0f\xa2\xfe\xeb\x82\x943\xecFl\x8e\x8e\x0bb\xf8\x7f\xa3[!\x95\x1e\xc5\\\t\xfe\x8ei\xea3\xe5\x904\xdd\x89QLx\xb5b\xce\xfd\xb9\xdaT\xf2\xbc\xf8\x85\xa9\xed`\x81ATP\x06\x8a"V-\xb0w\xec2Iv\xc0\x82R]\xe3\xdbK\t~[(\xa3\xd5z^Z\xdb\xc2"\xb1\xa3\xa42aaI#\x95\xf4\xc4|\xcbj84\xfb\xb7\xa21\x13\x996\x0e\x02Ju\x93\xe5\xe2\x86p:h\rEnT*\xd8\x16U\xd2\xc1\x91<\x81\xb4\x91\x1c\x88\xf0\xa7\xd1\xd1A9g}\x94\xfa+\x9f\x94\x0eg\x0c\xf1)\xe6Mw^B\xfd\xa0\xaa\x19RT\xc4v\x80^28\xd5\x85F\xa8sQw`\x962U{\xd0\xe0B\xbb\xd6<v\x0c\xe9\xf6\xaf\xab\xaf\xd1\xf1z\x9b\xb4E\x89!0\x86\x04U'


#### Desencriptado con clave privada

Una vez recibido el mensaje procedemos a desencriptarlo para ver su contenido.


In [3]:
privKeyFile = open("claveCE.ppk", "rb")
privateKey = serialization.load_pem_private_key(privKeyFile.read(), password=None, backend=default_backend())

pad = padding.OAEP(mgf=padding.MGF1(algorithm=hashes.SHA256()), algorithm=hashes.SHA256(), label=None)
msjRecuperado = privateKey.decrypt(msjEncriptado, pad)

print(msjRecuperado.decode())


Saben que ya será inútil volver, que se les ha degradado la memoria y que de ahora en adelante vivirán como fantasmas parciales que no dejan huellas de sus pasos y carecen de sombra.
